### How many games does a team needs to win to be crowned champions?

First we will need to import the standing tables

In [ ]:
import numpy as np
import pandas as pd
my_list_first =pd.read_html("https://fbref.com/en/comps/9/2021-2022/")
my_list = my_list_first.copy()

In [ ]:
squad_std_stats = my_list[2]
cols = squad_std_stats.columns
index = np.array(squad_std_stats['Unnamed: 0_level_0', 'Squad'])
squad_std_stats.head()

In [ ]:
# df_std = squad_std_stats[unnamed_cols].droplevel(0, axis=1)
squad_std_stats.index = index
df_pl_time = squad_std_stats["Playing Time"]
df_pl_time.head()

In [ ]:
from library import edit_standard_stats_table

In [ ]:
df_list = edit_standard_stats_table(my_list[2], my_list[3])

In [ ]:
col_0 = [["Performance", "Penalty Kicks"], [ "Goals", "Expected", "Launched", "Passes", "Goal Kicks", "Crosses", "Sweeper"]]


In [ ]:
gk_df = my_list[4]
index = np.array(gk_df["Unnamed: 0_level_0", "Squad"])
gk_df.index = index
gk_df = gk_df[col_0[0]]
gk_df = gk_df.droplevel(0, axis=1)
gk_df.drop(columns=["W", "D", "L"], inplace=True)

In [ ]:
from library import _edit_opp_df
gk_opp_df = my_list[5].copy()
gk_opp_df = _edit_opp_df(gk_opp_df)
gk_opp_df.index = index
gk_opp_df = gk_opp_df[col_0[0]]
gk_opp_df = gk_opp_df.droplevel(0, axis=1)
gk_opp_df.drop(columns=["W_Opp", "D_Opp", "L_Opp"], inplace=True)
gk_overall = gk_df.join(gk_opp_df)

In [ ]:
gk_overall.columns